### Create Color Magnitude Diagram of M34 
* Julia Frothingham, Alyssa Guzman, Molly Loughney, Jingyi Zhang
* 2021-11-26

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits 
from astropy.visualization import ZScaleInterval
import matplotlib.cm as cm
import glob 
import os
import pandas as pd
import scipy.ndimage.interpolation as interp
from matplotlib import colors

from astropy.stats import mad_std
from astropy.stats import sigma_clip
from photutils.utils import calc_total_error
import astropy.stats as stat
from photutils import aperture_photometry, CircularAperture, CircularAnnulus, DAOStarFinder

In [ ]:
import ast337_data_processing as dp
import ast337_photometry_analysis as pa

In [ ]:
#for everything
all_data_path = '/Users/mloughney/Desktop/2021oct28'
filters = ['B','V','R','I']

#for data reduction
biases = 'cal*bias.fit'
darks = 'cal*dark*'
flats = 'cal*flat*'
sta_star = 'sa115*'
M34 = 'M34*'

#for image alignment
standard_star_name = '{}Standard'
standard_star = 'fdb_sa115*.fit'
standard_star_ref = np.array([[[1895,1670],[1895,1550]]])
m34_name = '{}M34'
m34 = 'fdb_M34*.fit'
m34_ref = np.array([[[1488,1068],[1526,1134]]])

In [ ]:
#reduce standard star
dp.data_reduction (all_data_path, #path to directory with all images, including data and calibrations(str)
                    biases,       #naming format of bias frames (str)
                    darks,        #naming format of dark frames (str)
                    flats,        #naming format of flat fields (str)
                    sta_star,        #naming format of data
                    filters)      #list of filters

In [ ]:
#reduce M34 frames
dp.data_reduction (all_data_path, #path to directory with all images, including data and calibrations(str)
                    biases,        #naming format of bias frames (str)
                    darks,         #naming format of dark frames (str)
                    flats,         #naming format of flat fields (str)
                    M34,           #naming format of data
                    filters)       #list of filters

In [ ]:
#stack and align standard stars
dp.image_alignment (all_data_path,         # absolute path to the directory that contains all images
                     standard_star_name,    # naming format of desired stacked images
                     standard_star,         # naming format of the images that needs to be stacked
                     standard_star_ref,    # lists of refernce star and back ground pairs, in the format [[[ref1x,ref1y],[bg1x,bg1y]],[[ref2x,ref2y],[bg2x,bg2y]]]
                     filters,              #list of filters
                    )

In [ ]:
#stack M34
dp.image_alignment (all_data_path,         # absolute path to the directory that contains all images
                     m34_name,            # naming format of desired stacked images
                     m34,                  # naming format of the images that needs to be stacked
                     m34_ref,              # lists of refernce star and back ground pairs, in the format [[[ref1x,ref1y],[bg1x,bg1y]],[[ref2x,ref2y],[bg2x,bg2y]]]
                     filters,              #list of filters
                    )

In [ ]:
#checked the image
stacked = fits.getdata('s_StackedIStandard.fit')
plt.figure(figsize = (10,10))
plt.imshow(stacked,cmap = 'gray')#, norm = colors.LogNorm())
plt.colorbar()
plt.clim(0,100)

In [ ]:
#list of frames that needs photometry
sa_frames = [['s_StackedVStandard.fit', 9.91, 16, 28, 42, 'V'],
             ['s_StackedBStandard.fit', 18, 20, 35, 53, 'B'],
             ['s_StackedRStandard.fit', 8.01, 15, 27, 41, 'R'],
             ['s_StackedIStandard.fit', 7.58, 14, 25, 38, 'I'],]
m34_frames = [['s_StackedBM34.fit', 8.24, 14, 25, 38, 'B'],
             ['s_StackedVM34.fit', 10.91, 15, 27, 41, 'V'],
             ['s_StackedRM34.fit', 8.84, 14, 25, 38, 'R'],
             ['s_StackedIM34.fit', 9.30, 14, 25, 38, 'I'],]
sigma_limit = 7

In [ ]:
#aperture phtometry on standard star
sa_flux_table = pa.ast337_aperture_photometry (all_data_path,       #absolute path to all frames
                                                'StandardStarPhotometry',           # name of this object, used to store all photometry data in an CSV file
                                                15,          #number of sigmas that requires to identify a source
                                                sa_frames,    #list of images that wish to be measure, with their associate parameters in the form of 
                                                                  #[[name1, fwhm2, apertureradius2, sky ineer radius2, sky outer radius2, filter1],
                                                               #[name2, fwhm2, apertureradius2, sky ineer radius2, sky outer radius2, filter2]]
                                                   )

In [ ]:
#aperture phtometry on m34
m34_flux_table = pa.ast337_aperture_photometry (all_data_path,       #absolute path to all frames
                                                'M34Photometry',           # name of this object, used to store all photometry data in an CSV file
                                                sigma_limit ,          #number of sigmas that requires to identify a source
                                                m34_frames,    #list of images that wish to be measure, with their associate parameters in the form of 
                                                                  #[[name1, fwhm2, apertureradius2, sky ineer radius2, sky outer radius2, filter1],
                                                               #[name2, fwhm2, apertureradius2, sky ineer radius2, sky outer radius2, filter2]]
                                                   )

In [ ]:
# calculate the zeropoint in each filter 
standardstar = sa_flux_table.loc[[11]]
magzp_V = 9.695 - float(standardstar['V_inst'])
magzp_V_error = np.sqrt((float(standardstar['V_inst_err']))**2 + (0.0005)**2)
magzp_B = 9.695 + 0.615 - float(standardstar['B_inst'])
magzp_B_error = np.sqrt((float(standardstar['B_inst_err']))**2 + (0.0005)**2)
magzp_R = 9.695 - 0.353 - float(standardstar['R_inst'])
magzp_R_error = np.sqrt((float(standardstar['R_inst_err']))**2 + (0.0005)**2)
magzp_I = 9.695 - 0.353 - 0.349 - float(standardstar['I_inst'])
magzp_I_error = np.sqrt((float(standardstar['I_inst_err']))**2 + (0.0005)**2)

zp_parameter = [['V',magzp_V,magzp_V_error],
                ['B',magzp_B,magzp_B_error],
                ['R',magzp_R,magzp_R_error],
                ['I',magzp_I,magzp_I_error]]

In [ ]:
# calculate the "true magnitude" of objects in m34
m34_mag = pa.zero_calculation (zp_parameter,   #list of parameters of zero points in differnt filters, in the form of 
                                               #[[filter1, zeropoint magnitude1, zeropoint err2],
                                               # [filter2, zeropoint magnitude2, zeropoint err2]]
                                  m34_flux_table,      # the data frame of the sky that we want to measure, not the standard star
                                  'M34Mag')     #name of the chart for saving

In [ ]:
# calculate color index
VR = m34_mag['Vmag']- m34_mag['Rmag']
VRerr = np.sqrt((m34_mag['Vmag_err']) **2 + (m34_mag['Rmag_err'])**2)

In [ ]:
# plot the CMD
fig, (ax1) = plt.subplots(1, 1)
ax1.errorbar(VR, m34_mag['Vmag'], VRerr, m34_mag['Vmag_err'], marker = '+', linestyle='None')
ax1.set_ylabel('V mag')
ax1.set_xlabel('V-R')
ax1.set_title('CMD')
ax1.invert_yaxis()

In [ ]:
#isochrone fitting
dp.Isochrones_fitting ("M34Mag.csv",  # name of the file that contains the aparent magnitude of cluster stars
                        "M34",   # name of the cluster
                        "Lab10Materials/isochrones_marigo08_3e8yr.txt",  # name of the model isochrone
                        '300M',       # age of the model 
                        ['B','V'],     # a list of two filters that used for calculating the color index,
                                        # eg. ['V', 'R'] then V-R would be on the xaxis of the CMD
                        'V',          # the filter that the color index should be plotted against
                        8.5# the shift in the yaxis  so that the isochrone matches the actual data
                      )